  ## Libraries

In [62]:
# IMPORT THESE FIRST.
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.datasets import (make_classification, make_blobs, load_breast_cancer, make_regression, make_friedman1,
                              load_iris)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_val_score, validation_curve, train_test_split

In [18]:
df = pd.read_excel("https://github.com/TechLabs-Berlin/wt21-pet-match/blob/main/data_set/pet-set.xlsx?raw=true")
#df.head()

### variable discribing the satisfaction level in percentage

In [19]:
# replacing strings with numerical values
df['expectations'] = df['My cat has met all my expectations'].replace({'Strongly agree': 5, 'Agree': 4, 'Neither agree nor disagree': 3, 'Disagree': 2, 'Strongly disagree': 1})
df['happy_owner'] = df['I am happy with my cat'].replace({'Strongly agree': 5, 'Agree': 4, 'Neither agree nor disagree': 3, 'Disagree': 2, 'Strongly disagree': 1})
df['happy_cat'] = df['I feel my cat is happy living with me'].replace({'Strongly agree': 5, 'Agree': 4, 'Neither agree nor disagree': 3, 'Disagree': 2, 'Strongly disagree': 1})
df['rehome'] = df['I have considered relinquishing or rehoming this cat to someone else'].replace({'Strongly agree': 1, 'Agree': 2, 'Neither agree nor disagree': 3, 'Disagree': 4, 'Strongly disagree': 5})

# aggregating final score
df['satisf_level'] = (df['expectations'] + df['happy_owner'] + df['happy_cat'] + df['rehome'])/20

# Owners / Adopters

The aggregarion was made by Hany, all comments also belong to him:)
Personality traits grouped in five scores according to BFI:

* Extraversion

* Agreeableness

* Conscientiousness

* Neuroticism  (how tense and moody the person )

* Creativity

In [20]:
# slice with the BFI relevant adopters data
adopters = df.iloc[:, 61:]
# remove the NaN , they are text here
adopters.replace("NaN",np.nan, inplace =True)
adopters.fillna(0,inplace=True)

# lisitng the Reverse questions (take care of the numbering)
rev_col = adopters.iloc[:,[1,5,7,8,11,17,20,22,23,26,30,33,34,36,40,42]]
rev_ques = rev_col.replace({'Agree strongly': 1, 'Agree a little': 2, 'Neither agree nor disagree': 3, 'Disagree a little': 4, 'Disagree strongly': 5})

#listing normal questions 
normal_ques = adopters.drop(columns = rev_col).replace({'Agree strongly': 5, 'Agree a little': 4, 'Neither agree nor disagree': 3, 'Disagree a little': 2, 'Disagree strongly': 1})

# make them into one set, 
adopt_score =  pd.concat([ normal_ques, rev_ques], axis=1)

#adopt_score.head()

#### note: the code may require some refactoring (working with the bigger set )
- **used "conctinate instade of inplace= True** : with .replace it gets warning " copy of a slice from a DataFrame" and the inplace is not fully happpening espically when applaying that for the 2nd time 

#### clastering (seems little hard coded, due to the use of concatinate)

- rules (all them are re-aggranged as follows)
Extraversion:(8 questions) 1, 6R, 11, 16, 21R, 26, 31R, 36
Agreeableness: (9 questions)2R, 7, 12R, 17, 22, 27R, 32, 37R, 42 ## high no = high agreebleness 
Conscientiousness: (9 questions)3, 8R, 13, 18R, 23R, 28, 33, 38, 43R
Neuroticism:(8 questions) 4, 9R, 14, 19, 24R, 29, 34R, 39
Openness: (10 ques )5, 10, 15, 20, 25, 30, 35R, 40, 41R, 44 

In [21]:
# arrange them for better reading.


extraversion = adopt_score.iloc[:,[0,29,6,10,34,16,38,22]]          ##Extraversion

agreeableness = adopt_score.iloc[:,[28,4,32,11,14,37,20,41,26]]     ##Agreeableness

conscientiousness = adopt_score.iloc[:,[1,30,7,33,35,17,21,23,43]]  ##Conscientiousness

neuroticism = adopt_score.iloc[:,[2,31,8,12,36,18,39,24]]                 ##Neuroticism  (how tense and moody the person )

creativity = adopt_score.iloc[:,[3,5,9,13,15,19,40,25,42,-1]]      ## openess & creativity


## putting them into columns with precenatges (it can be added to any df )

adopt_score["extraversion"] =extraversion.sum(axis=1)/(len(extraversion.columns)*5)
adopt_score["easy_going"] =agreeableness.sum(axis=1)/(len(agreeableness.columns)*5)
adopt_score["diligent"] =conscientiousness.sum(axis=1)/(len(conscientiousness.columns)*5)
adopt_score["moody_tense"] =neuroticism.sum(axis=1)/(len(neuroticism.columns)*5)
adopt_score["creative"] =creativity.sum(axis=1)/(len(creativity.columns)*5)

adopt_scores_only = adopt_score[["extraversion", "easy_going", "diligent", "moody_tense", "creative"]]

In [22]:
#adopt_scores_only.head(3)

# Cats

Grouping cats into 4 personalitiy-Cluster performed by Dayo, I changed it a bit so that we get comparable scores for cats and adopters.

* PC1: Gregariousness (confident, friendly, interactive

* PC: Aggressiveness (agonistic when handled or interacted with)

* PC: Aloofness/avoidance(avoidant of interaction, not friendly

* PC4: Anxiousness/fearfulness (wary of novel/less familiarpeople)


In [23]:
cats_pc = df.iloc[:, 26 : 54]
# remove the NaN , they are text here
cats_pc.replace("NaN",np.nan, inplace =True)
cats_pc.fillna(0,inplace=True)

# listing the Reverse questions (take care of the numbering)
rev_col_c = cats_pc.iloc[:,[9,16,18,19,20,27]].rename(columns = {"My cat comes and asks me for attention and initiates contact with me (e.g. the cat comes and sits on my knee, or rubs up against me and around me, in order to receive fuss/strokes/chin/cheek tickles)": 'REVERSED My cat comes and asks me for attention and initiates contact with me',
                                          'My cat is timid': 'REVERSED My cat is timid',
                                          'My cat likes being stroked': 'REVERSED My cat likes being stroked',
                                          'My cat is very tolerant to being handled': 'REVERSED My cat is very tolerant to being handled',
                                          'My cat is fearful': 'REVERSED My cat is fearful',
                                          'My cat would prefer to be left alone, rather than be with people': 'REVERSED My cat would prefer to be left alone, rather than be with people'})
rev_ques_c = rev_col_c.replace({'Strongly disagree': 5, 'Never': 5, 'Disagree': 4, 'Once': 4, 'Neither agree nor disagree': 3, 'Occasionally': 3, 'Agree': 2, 'Usually': 2, 'Strongly agree': 1, 'Always': 1})

#listing normal questions 
normal_ques_c = cats_pc.replace({'Strongly disagree': 1, 'Never': 1, 'Disagree': 2, 'Once': 2, 'Neither agree nor disagree': 3, 'Occasionally': 3, 'Agree': 4, 'Usually': 4, 'Strongly agree': 5, 'Always': 5}).drop(columns=cats_pc.iloc[:, [18, 19, 22]])

# make them into one set, 
cats_score =  pd.concat([normal_ques_c, rev_ques_c], axis=1)

# grouping
gregariousness = cats_score.iloc[:,[0,1,2,9,15,17,18,23,24,26,27]] #Gregariousness (confident, friendly, interactive)
aggressiveness = cats_score.iloc[:,[3,7,10,11,12,13,29]] #Aggressiveness (agonistic when handled or interacted with)
avoidance = cats_score.iloc[:,[4,5,6,8,14,19,25,28]] #Aloofness/avoidance(avoidant of interaction, not friendly)
anxiousness = cats_score.iloc[:,[16,20,21,22,24,]] #Anxiousness/fearfulness (wary of novel/less familiarpeople)

# aggregating to percentages
cats_score["friendly"] =gregariousness.sum(axis=1)/(len(gregariousness.columns)*5)
cats_score["agressive"] =aggressiveness.sum(axis=1)/(len(aggressiveness.columns)*5)
cats_score["avoidant"] =avoidance.sum(axis=1)/(len(avoidance.columns)*5)
cats_score["anxious"] =anxiousness.sum(axis=1)/(len(anxiousness.columns)*5)

cats_scores_only = cats_score[["friendly","agressive", "avoidant", "anxious"]]

In [24]:
df_scores = pd.concat([cats_scores_only, adopt_scores_only, df["satisf_level"]], axis=1)
#df_scores.head(3)

# Three datasets with scores only:

* adopt_scores_only: personality scores of potential adopters
* cats_scores_only: cats' personality scores
* df_scores: cats' and owners' scores + satisfaction level

In [25]:
adopt_scores_only.head(5)

,extraversion,easy_going,diligent,moody_tense,creative
0,0.675,0.822222,0.711111,0.700,0.64
1,0.675,0.800000,0.800000,0.750,0.78
2,0.625,0.418889,0.466667,0.600,0.28
3,0.575,0.800000,0.577778,0.925,0.72
4,0.575,0.733333,0.666667,0.675,0.58


In [26]:
cats_scores_only.head(5)

,friendly,agressive,avoidant,anxious
0,0.690909,0.314286,0.650,0.68
1,0.654545,0.571429,0.475,0.60
2,0.654545,0.514286,0.700,0.64
3,0.727273,0.371429,0.450,0.84
4,0.709091,0.342857,0.550,0.76


In [27]:
df_scores.head(5)

,friendly,agressive,avoidant,anxious,extraversion,easy_going,diligent,moody_tense,creative,satisf_level
0,0.690909,0.314286,0.650,0.68,0.675,0.822222,0.711111,0.700,0.64,1.00
1,0.654545,0.571429,0.475,0.60,0.675,0.800000,0.800000,0.750,0.78,1.00
2,0.654545,0.514286,0.700,0.64,0.625,0.418889,0.466667,0.600,0.28,0.85
3,0.727273,0.371429,0.450,0.84,0.575,0.800000,0.577778,0.925,0.72,1.00
4,0.709091,0.342857,0.550,0.76,0.575,0.733333,0.666667,0.675,0.58,1.00


In [48]:
df_scores.isna().value_counts()

friendly  agressive  avoidant  anxious  extraversion  easy_going  diligent  moody_tense  creative  satisf_level
False     False      False     False    False         False       False     False        False     False           3248
                                                                                                   True              83
dtype: int64

# Clustering KNN

In [28]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X - cats_scores_only

In [51]:
df_1 = df_scores.dropna()

In [52]:
x_cats = df_1[['friendly', 'agressive', 'avoidant', 'anxious']]
y_cats = df_1['satisf_level']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(x_cats, y_cats, random_state=0)

In [54]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
# we must apply the scaling to the test set that we computed for the training set
X_test_scaled = scaler.transform(X_test)

In [55]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures


In [58]:
knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X_train, y_train)